# Multi-Agent Systems with AutoGen

Welcome to this workshop on multi-agent systems using [AutoGen](https://github.com/microsoft/autogen)! In this notebook, we'll explore how to create, configure, and orchestrate multiple agents to collaborate in solving complex tasks.

## What You'll Learn
- How to transition from single agents to multi-agent systems
- The fundamentals of GroupChat for agent collaboration
- How to design specialized agents with different roles
- Strategies for managing agent conversations and turn-taking
- Techniques for controlling conversation flow and termination

Let's start by setting up our environment.

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import StructuredMessage, TextMessage
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

import os 
from dotenv import load_dotenv
load_dotenv(override=True)  # Load environment variables from .env file

# Load environment variables
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
model_name = os.getenv("AZURE_OPENAI_MODEL_NAME")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
base_url = os.getenv("AZURE_OPENAI_ENDPOINT")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")

### Multiple Agent Types and Roles

In a multi-agent system, each agent should have a clearly defined role and purpose. Here are some common agent roles you might implement:

| Role Type | Purpose | Example Instructions |
|-----------|---------|---------------------|
| **Researcher** | Provides factual information | Focus on accurate information with sources |
| **Creative** | Generates novel ideas | Think outside conventional boundaries |
| **Critic** | Analyzes and critiques | Identify weaknesses and suggest improvements |
| **Summarizer** | Condenses information | Extract key points concisely |
| **Moderator** | Guides conversation | Keep discussion on track and productive |
| **Domain Expert** | Provides specialized knowledge | Share deep expertise in a specific field |

Let's create a more sophisticated team of agents with specialized roles:

In [ ]:
# Create a team of specialized agents
az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=deployment_name,
    model=model_name,
    api_version=api_version,
    azure_endpoint=base_url,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=api_key, # For key-based authentication.
)

# 1. Researcher who provides factual information
researcher = AssistantAgent(
    name="Researcher",
    model_client=az_model_client,
    system_message="""You are a thorough researcher who provides factual information and analysis.
    
    Your responsibilities:
    - Provide accurate, factual information on the topic
    - Analyze questions from multiple angles
    - Consider historical context and current understanding
    - Be objective and balanced in your assessment
    - Acknowledge limitations in current knowledge when appropriate
    
    Always strive for accuracy over speculation. Keep your responses very concise, clear and straightforward.""",
)

# 2. Creative thinker who generates innovative ideas
innovator = AssistantAgent(
    name="Innovator",
    model_client=az_model_client,
    system_message="""You are an innovative thinker who generates novel ideas and perspectives.
    
    Your responsibilities:
    - Suggest unique approaches and solutions
    - Think beyond conventional boundaries
    - Make unexpected connections between concepts
    - Offer imaginative scenarios and possibilities
    - Propose 'what if' scenarios to expand thinking
    
    Don't be constrained by traditional thinking - be bold and creative. Keep your responses very concise, imaginative and engaging.""",
)

# 3. Critic who evaluates ideas critically
critic = AssistantAgent(
    name="Critic",
    model_client=az_model_client,
    system_message="""You are a thoughtful critic who evaluates ideas and identifies potential issues.
    
    Your responsibilities:
    - Analyze the strengths and weaknesses of proposals
    - Identify potential problems or limitations
    - Challenge assumptions constructively
    - Suggest improvements to ideas
    - Consider practical implementation challenges
    
    Be constructive in your criticism - your goal is to improve ideas, not dismiss them. Keep your responses very concise, clear and straightforward.""",
)

# 4. Synthesizer who brings ideas together
synthesizer = AssistantAgent(
    name="Synthesizer",
    model_client=az_model_client,
    system_message="""You are a skilled synthesizer who integrates diverse perspectives into coherent conclusions.
    
    Your responsibilities:
    - Identify common themes across different viewpoints
    - Reconcile apparently conflicting ideas when possible
    - Create a balanced, integrated perspective
    - Summarize key points from the discussion
    - Draw reasonable conclusions from the collective input
    
    Your goal is to bring together different perspectives into a coherent whole. Keep your responses very concise, clear and straightforward.""",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

# Create a group chat with all specialized agents
expert_team_chat = RoundRobinGroupChat(
    [researcher, innovator, critic, synthesizer], 
    termination_condition=text_termination,
    max_turns=10,
)

Now let's test our expert team with the same climate change question to see how their specialized roles affect the collaboration:

In [ ]:
question = "What are some approaches to address climate change?"

In [ ]:
# Test the expert team with the same question
await Console(expert_team_chat.run_stream(task=question))  # Stream the messages to the console.

Compare the two approaches:

1. **Simple Group Chat (2 agents)**:
   - Basic factual and creative perspectives
   - Limited interaction depth
   - Simple back-and-forth exchange

2. **Expert Team (4 specialized agents)**:
   - Multiple specialized perspectives
   - More comprehensive analysis
   - Natural progression from facts → ideas → critique → synthesis

This demonstrates how designing agents with complementary roles can create more valuable interactions and outputs.

### Exercise: Create Your First Multi-Agent Conversation

Now it's your turn to design a multi-agent system! Create a team of 3-4 agents with different, complementary roles to collaborate on a specific type of problem.

Your task:
1. Create 3-4 agents with distinct roles and specialties
2. Create a GroupChat with these agents
3. Test the multi-agent system with a relevant question or task
4. Observe how the agents work together and complement each other

In [ ]:
# Your code here

# 1. Create 3-4 specialized agents
# (Define each agent with a clear role and detailed instructions)


# 2. Create a GroupChat with your agents
# (Set up an appropriate max_round and manager)


# 3. Define a relevant question or task for your team


# 4. Run the group chat and observe the interaction

<details>
<summary>Click to see solution</summary>

```python
# 1. Create specialized agents for product development
az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=deployment_name,
    model=model_name,
    api_version=api_version,
    azure_endpoint=base_url,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=api_key, # For key-based authentication.
)

# Product Manager focuses on user needs and market fit
product_manager = AssistantAgent(
    name="ProductManager",
    model_client=az_model_client,
    system_message="""You are a strategic product manager focused on user needs and market fit.
    
    Your responsibilities:
    - Identify user problems and needs
    - Evaluate market opportunities and competition
    - Define product requirements and priorities
    - Consider business viability and market fit
    - Focus on user experience and value proposition
    
    Always center your thinking on user problems and how to solve them effectively."""
)

# Engineer focuses on technical implementation
engineer = AssistantAgent(   
    name="Engineer",
    model_client=az_model_client,
    system_message="""You are a technical engineer who focuses on implementation feasibility and technical solutions.
    
    Your responsibilities:
    - Evaluate technical feasibility of proposed features
    - Suggest appropriate technologies and architectures
    - Identify potential technical challenges and limitations
    - Consider scalability, performance, and security implications
    - Propose practical implementation approaches
    
    Think in terms of practical implementation and technical constraints."""
)

# Designer focuses on user experience
designer = AssistantAgent(    
    name="Designer",
    model_client=az_model_client,
    system_message="""You are a user-centered designer who ensures products are usable and delightful.
    
    Your responsibilities:
    - Consider user interactions and experience flows
    - Evaluate accessibility and usability concerns
    - Suggest intuitive interfaces and interactions
    - Think about visual design principles and consistency
    - Address potential user pain points and confusion
    
    Always advocate for the end user's experience and needs."""
)

# Marketing specialist focuses on positioning and communication
marketer = AssistantAgent(
    name="Marketer",
    model_client=az_model_client,
    system_message="""You are a marketing specialist who thinks about how to position and promote products.
    
    Your responsibilities:
    - Consider how to position the product in the market
    - Identify target audiences and messaging strategies
    - Suggest go-to-market approaches and channels
    - Think about brand alignment and messaging
    - Address potential objections or barriers to adoption
    
    Focus on how to communicate value and drive adoption of the product."""
)

# 2. Create a product development team chat
product_team = RoundRobinGroupChat(
    [product_manager, engineer, designer, marketer], 
    max_turns=8, # 2 rounds of all 4 agents
)

# 3. Define a product development challenge
product_challenge = "We need to develop a mobile app that helps people track and reduce their carbon footprint in daily life. What approach should we take?"

# 4. Run the product team discussion
await Console(product_team.run_stream(task=product_challenge))  # Stream the messages to the console.

# Analyze how the agents complemented each other
print("\nMulti-Agent Collaboration Analysis:")
print("- The Product Manager focused on user needs and market positioning")
print("- The Engineer addressed technical feasibility and implementation approaches")
print("- The Designer considered user experience and interface design")
print("- The Marketer thought about target audience and adoption strategy")
print("- Together, they provided a comprehensive product development approach")
```
</details>

## 2. Introduction Agent Strategies

In the previous section, we created basic multi-agent systems using `GroupChat` with default strategies. Now let's explore how to build more sophisticated agent collaborations by configuring agents with specialized roles and customizing their interactions.

### Initializing Multiple Specialized Agents

When building effective multi-agent systems, the way you design and initialize your agents is critical. Each agent should have:

1. **Clear purpose**: A well-defined role and responsibility
2. **Specialized instructions**: Guidance on how to approach problems from its unique perspective
3. **Complementary skills**: Abilities that work well with other agents' capabilities
4. **Communication style**: How it should present information to other agents

Let's create a team of specialized agents for a specific task: collaborative writing and editing.

In [ ]:
az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=deployment_name,
    model=model_name,
    api_version=api_version,
    azure_endpoint=base_url,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=api_key, # For key-based authentication.
)

# Create a writer agent that generates content
writer = AssistantAgent(
    name="Writer",
    model_client=az_model_client,
    system_message="""You are a creative writer who crafts engaging content.
    
    Your role in this collaboration:
    1. Generate original content based on the topic provided
    2. Apply creative storytelling techniques to engage readers
    3. Incorporate feedback from the editor and fact-checker to improve your writing
    4. Revise content to address issues raised by other team members
    5. Focus on creating a compelling narrative voice and structure
    
    When responding to feedback:
    - Be open to constructive criticism
    - Explain your creative choices when relevant
    - Incorporate suggestions that improve the content
    
    Always strive to maintain the core message while making the content more engaging and effective. Keep your responses very concise, imaginative and engaging.""",
)

# Create an editor agent that improves the writing
editor = AssistantAgent(
    name="Editor",
    model_client=az_model_client,
    system_message="""You are a meticulous editor who improves content quality and clarity.
    
    Your role in this collaboration:
    1. Review content for clarity, coherence, and flow
    2. Identify and fix grammatical, structural, or stylistic issues
    3. Suggest improvements to enhance readability and impact
    4. Ensure the content meets its intended purpose and audience needs
    5. Maintain consistent voice and tone throughout
    
    When providing feedback:
    - Be specific about what needs improvement and why
    - Offer constructive suggestions rather than just criticism
    - Consider both micro (sentence-level) and macro (structure) improvements
    - Balance preserving the writer's voice with improving the content
    
    Your goal is to elevate the writing while respecting the writer's intent and style. Keep your responses very concise, clear and straightforward.""",
)

# Create a fact-checker agent that ensures accuracy
fact_checker = AssistantAgent(
    name="FactChecker",
    model_client=az_model_client,
    system_message="""You are a thorough fact-checker who ensures accuracy and credibility.
    
    Your role in this collaboration:
    1. Verify factual claims in the content
    2. Identify potential inaccuracies or misleading statements
    3. Suggest corrections for any factual errors
    4. Recommend additional context where needed for accuracy
    5. Ensure the content is truthful and well-supported
    
    When providing feedback:
    - Focus on accuracy rather than style or structure
    - Explain why a statement might be problematic
    - Provide correct information to replace inaccuracies
    - Consider potential sources of factual support
    
    Your goal is to ensure the content maintains high standards of accuracy and integrity. Keep your responses very concise, clear and straightforward.""",
)

print("Created specialized writing team agents successfully!")

### Setting Up a Basic Group Chat

Now that we have our specialized agents, let's set up a `GroupChat` to orchestrate their collaboration. We need to consider:

1. **Agent order**: The sequence in which agents participate
2. **Maximum rounds**: How many turns the conversation should take

For this example, we'll use a simple sequential approach where agents take turns in a fixed order.

In [ ]:
# Create a writing team chat with sequential order
writing_team_chat = RoundRobinGroupChat(
    [writer, editor, fact_checker],
    max_turns=9,  # 3 rounds × 3 agents
)

The default GroupChatManager in AutoGen uses a round-robin pattern, moving through agents in the order they were added. It's predictable and ensures each agent gets a turn to contribute.

Now let's test our writing team collaboration with a writing task:

In [ ]:
# Test the writing team with a specific writing task
writing_task = "Write a short blog post about the impact of artificial intelligence on healthcare, focusing on recent advancements and ethical considerations."

# Run the group chat
await Console(writing_team_chat.run_stream(task=writing_task))

Did you notice the pattern in the conversation?

1. The **Writer** created the initial draft
2. The **Editor** reviewed it for clarity and style
3. The **FactChecker** verified the factual claims
4. The **Writer** revised based on feedback
5. The **Editor** reviewed the revised version
6. And so on...

This sequential approach mimics a real editorial process where content moves through different stages of review and revision.

### Simple Agent Collaboration Patterns

The sequential selection we just used is one of several agent collaboration patterns. Here are some common patterns you might use:

1. **Sequential (Round-Robin)**: Agents take turns in a fixed order
   - Pros: Simple, predictable, ensures each agent participates
   - Cons: Not responsive to content; might include irrelevant contributions

2. **Fixed Workflow**: Agents follow a specific sequence designed for a task
   - Pros: Mirrors real workflows (e.g., write → edit → approve)
   - Cons: Less flexible for unexpected situations

3. **Dynamic Selection**: An LLM decides which agent should respond next
   - Pros: More responsive to conversation needs; adaptable
   - Cons: Less predictable; requires careful prompt engineering

Let's implement a fixed workflow pattern for our writing team, where we explicitly define the sequence of agents based on the writing process:

In [ ]:
from typing import List, Sequence

from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage
from autogen_agentchat.teams import SelectorGroupChat


def candidate_func(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> List[str]:
    # keep planning_agent first one to plan out the tasks
    if messages[-1].source == "user":
        return [writer.name]

    # Define a specific writing workflow
    # Writer -> Editor -> FactChecker -> Writer (for revisions) -> Editor (final review)
    participants = []

    last_message = messages[-1]
    if last_message.source == writer.name:
        participants.append(editor.name)
    elif last_message.source == editor.name:
        participants.append(fact_checker.name)
    elif last_message.source == fact_checker.name:
        participants.append(writer.name)
    
    return participants

workflow_agents = [writer, editor, fact_checker]

workflow_chat = SelectorGroupChat(
    workflow_agents,
    max_turns=5,
    model_client=az_model_client,
    candidate_func=candidate_func,
)

Now let's test our fixed workflow collaboration with the same writing task:

In [ ]:
# Test the workflow chat with the same writing task
await Console(writing_team_chat.run_stream(task=writing_task))

The fixed workflow gives us more control over the exact sequence of agent interactions. This is especially useful for process-oriented tasks that have a natural sequence, like content creation and review.

Notice how we customized the workflow to match a realistic content creation process:
1. **Writer**: Creates initial draft
2. **Editor**: Reviews for style and clarity
3. **FactChecker**: Verifies accuracy
4. **Writer**: Revises based on all feedback
5. **Editor**: Gives final approval

This is more directed than the simple round-robin approach and can lead to a more efficient collaboration for specific tasks.

### Exercise: Implement a Two-Agent Interaction System

Now it's your turn to implement a specialized two-agent interaction system. Create two agents with complementary roles that can collaborate effectively on a specific task.

Your task:
1. Create two specialized agents with clear, complementary roles
2. Set up a GroupChat with an appropriate agent order
3. Define a specific problem or task for them to collaborate on
4. Test and demonstrate their interaction

In [ ]:
# Your code here

# 1. Create two specialized agents with complementary roles
# (Make sure their instructions are detailed and clearly define their responsibilities)


# 2. Set up a GroupChat with appropriate agent order and manager
# (Consider whether a sequential or custom order would work best)


# 3. Define a specific problem or task
# (Choose something that benefits from both agents' perspectives)


# 4. Test the interaction
# (Run the chat and observe how they collaborate)

<details>
<summary>Click to see solution</summary>

```python
# 1. Create two specialized agents for debating a topic
az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=deployment_name,
    model=model_name,
    api_version=api_version,
    azure_endpoint=base_url,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=api_key, # For key-based authentication.
)

proponent = AssistantAgent(
    name="Proponent",
    model_client=az_model_client,
    system_message="""You are a persuasive advocate who makes strong arguments in favor of a position.
    
    Your role in this debate:
    - Present compelling arguments supporting the given position
    - Use evidence, logic, and persuasive techniques to strengthen your case
    - Address counterarguments raised by the opponent
    - Focus on the strongest points in favor of your position
    - Maintain a respectful, fact-based approach
    
    When responding to the opponent:
    - Acknowledge their points before presenting counterarguments
    - Find weaknesses in their reasoning or evidence
    - Redirect the conversation to your stronger arguments
    
    Your goal is to make the most compelling case possible for your position."""
)

opponent = AssistantAgent(
    name="Opponent",
    model_client=az_model_client,
    system_message="""You are a critical thinker who makes strong arguments against a position.
    
    Your role in this debate:
    - Present compelling arguments challenging the given position
    - Identify weaknesses, limitations, or unintended consequences
    - Use evidence, logic, and critical analysis to strengthen your case
    - Address arguments raised by the proponent
    - Maintain a respectful, fact-based approach
    
    When responding to the proponent:
    - Acknowledge their strongest points
    - Identify logical fallacies or evidentiary gaps
    - Present alternative perspectives or solutions
    
    Your goal is to provide a balanced counterargument and critical evaluation."""
)

debate_chat = RoundRobinGroupChat(
    [proponent, opponent],
    max_turns=6,  # 3 exchanges each
)

debate_topic = "Topic for debate: Should social media platforms be responsible for moderating user content?"

print("=== Starting Structured Debate ===\n")
print("Format: Proponent and Opponent will alternately present their cases\n")

# Run the group chat
await Console(debate_chat.run_stream(task=debate_topic))
```
</details>

### Key Takeaways

In these sections, we've learned how to:

1. **Transition from single agents to multi-agent systems**
   - Understand when and why multiple agents are beneficial
   - Design agents with complementary roles and specializations

2. **Work with GroupChat**
   - Create and configure a basic group chat
   - Understand key components like agent order and max rounds
   - Run multi-agent conversations and process responses

3. **Implement different agent collaboration patterns**
   - Use sequential (round-robin) order
   - Create fixed workflow sequences
   - Design specialized agent teams for specific tasks

In the next section, we'll explore more advanced techniques for controlling multi-agent conversations, including custom agent selection and workflow strategies.

# Dev Container Improvements

This cell documents improvements made to the dev container setup for this repository.

## 1. requirements.txt File
Ensure that a `requirements.txt` file exists at the project root and includes all dependencies needed for your notebooks, such as `autogen`, `jupyterlab`, and any other libraries used in your code cells.

## 2. Python Interpreter Path
The `python.defaultInterpreterPath` in your devcontainer configuration is set to `/usr/local/bin/python`. This is correct for the default image, but if you change the base image or Python version, verify that this path matches the installed Python interpreter.

## 3. User Permissions
The Dockerfile creates a non-root user (`vscode`). Ensure all project files and folders are accessible to this user. If you encounter permission issues, add a `chown` command in the Dockerfile after copying files.

## 4. Jupyter Port Forwarding
Add the following to your `devcontainer.json` to automatically forward the Jupyter port:

```json
"forwardPorts": [8888]
```

## 5. Post-Create Command
If you already install requirements in the Dockerfile, you can remove the `postCreateCommand` from `devcontainer.json` to avoid duplicate installs. Alternatively, remove the install from the Dockerfile and keep the post-create command.

## 6. Optional: Pre-commit Hooks
For collaborative projects, consider adding pre-commit hooks for code formatting or linting, and install them in the Dockerfile.

## 7. Optional: Environment Variables
If your notebooks require environment variables (e.g., for Azure credentials), add a `.env.example` file and document the required variables for contributors.

## 8. Optional: VS Code Settings
You can add more settings in `devcontainer.json`, such as enabling notebook features or setting a default working directory for notebooks.

## 9. Optional: Extensions
If you use other tools (e.g., GitLens, Python Black), add their extensions to the list in `devcontainer.json` for a more complete dev environment.